# Validate Minian outputs

- Load
    - spatial map (A) -> maybe to associate with a projection of neuron fluorescence
    - temporal activity of detected neurons

- Validate units
- Extract and save relevant data for each selected unit
    - spatial location (x, y); no need for shape
    - Ca2+ trace
    - Ca2+ peak time and amplitude
    - Deconvolved spikes


## Load the required packages (and many more that are not useful)

In [ ]:

import numpy as np
import csv
from pathlib import Path
import xarray as xr
import pandas as pd
from pandas import concat
import matplotlib.pyplot as plt
import itertools as itt
import os
import sys
import json
from scipy import signal
from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

import holoviews as hv
import panel as pn
import param
from dask.distributed import Client, LocalCluster
from holoviews.operation.datashader import datashade, regrid
from holoviews.util import Dynamic
from holoviews import opts
from holoviews import Store
from holoviews.operation.datashader import shade
hv.extension('bokeh', 'matplotlib')
#from IPython.core.display import display
from IPython.display import display
from ipyfilechooser import FileChooser



#%reset

In [ ]:
# Rq pour Aurélie: tu n'as pas besoin de cd dans le repertory pour importer ses modules ; il suffit d'ajouter son chemin au path pour cela.
# Si on doit travailler avec des fichiers qui peuvent convenir de partout (au NC, chez moi et à Cambridge), il serait très
# pratique de s'en tenir à des chemins relatifs. Ici, le dossier minian est cherché dans le dossier grand-parent de ton script.
# ( les '..' veulent dire 'remonte d'un dossier' ; '.' voudrait dire 'le dossier actuel')
# Si le dossier minian est à C:/Users/Manip2/SCRIPTS/Code python audrey/code python aurelie/interfaceJupyter/minian, ton script
# devrait être à C:/Users/Manip2/SCRIPTS/Code python audrey/code python aurelie/interfaceJupyter/{PeutImporteCeNom}/10_validate...
# On peut changer la structure pour une qui te convient mieux mais faudrait qu'on trouve un truc auquel on se tient
minian_path = os.path.join(os.path.abspath('..'),'minian')
print("The folder used for minian procedures is : {}".format(minian_path))

sys.path.append(minian_path)
from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)

## Import the minian files

### Select the folder

In [ ]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
except:
    print("the path was not defined in store")
    #dpath = "/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/ExampleRedLines/2022_08_06/13_30_01/My_V4_Miniscope/"
    dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Folder with videos</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)



### Import spatial map, Ca2+ traces, deconvolved spikes

In [ ]:
minianversion = 'minianAB' # or 'minian_intermediate'

folderMouse = Path(os.path.join(dpath,minianversion))
print(folderMouse)
minian_ds = open_minian(folderMouse)

A = minian_ds['A']
C = minian_ds['C']
S = minian_ds['S']

B = A['unit_id']
series = B.to_series()
D = series.count()

idloc = A.idxmax("unit_id")
Hmax = A.idxmax("height")
Hmax2 = Hmax.max("width")

Wmax = A.idxmax("width")
Wmax2 = Wmax.max("height")
coord1 = Wmax2.to_series()
coord2 = Hmax2.to_series()

a = pd.concat([coord1,coord2], axis=1)
unit = len(a)
print("{} units have been found".format(unit))

## Plot either all cells or just the one of interest

### Define the actions triggered by drop/keep buttons

### Plot the spatial map for all cells + interactive Ca2+ trace

In [ ]:

output_size = 80
hv.output(size=int(output_size))
opts = dict(
    plot=dict(colorbar=True, invert_yaxis=True),
    style=dict(cmap="Viridis"),
)
image = hv.Image(
    A.max("unit_id").compute().astype(np.float32).rename("A"),
    kdims=["width", "height"],
)#.opts(**opts)

#labels = hv.Labels([(a.iloc[i,0], a.iloc[i,1], a.index[i]) for i in range(5)]) #np.arange(50,100,1)
labels = hv.Labels([(a.iloc[i,0], a.iloc[i,1], a.index[i]) for i in range(len(a))]).opts(text_color='orange',  text_font_size='9pt')

plot_unit = hv.HoloMap({a.index[i]: hv.Curve((C[i,:]), group='keep') for i in range(len(a))}, kdims='Value').opts(ylim=(-0.5, 20)) #np.arange(50,100,1)

class ActionSelect(param.Parameterized):
    actionDrop = param.Action(default=lambda x: x.param.trigger('actionDrop'), label='Click here to drop!')
    actionKeep = param.Action(default=lambda x: x.param.trigger('actionKeep'), label='Click here to keep!')

    unit_to_keep=a.index.tolist()

    @param.depends('actionDrop')
    def drop_unit(self):
        number = layout[1][1][0].value
        if number in self.unit_to_keep:
            self.unit_to_keep.remove(number)
           
    @param.depends('actionKeep')
    def keep_unit(self):
        number = layout[1][1][0].value
        if number not in self.unit_to_keep:
            self.unit_to_keep.append(number)
            self.unit_to_keep.sort()


selection = ActionSelect()

layout = pn.Row(image * labels, plot_unit, pn.Column(
        pn.panel(selection, show_name=False, margin=0, widgets={"actionDrop": {"button_type": "primary"}, }),
    selection.drop_unit, selection.keep_unit,
    )
    )
display(layout)



Feel unit to keep

In [ ]:
# fill in unit_to_kepp with cell_id of cells to keep 

print("The units kept are:")
print(selection.unit_to_keep)

all_units=a.index.tolist()
unit_to_drop = [e for e in all_units if e not in selection.unit_to_keep]
print("The units dropped are:")
print(unit_to_drop)


Remove dropped units

In [ ]:
copyB = list(B.copy())
for i in range(len(unit_to_drop)):
    elem = unit_to_drop[i]
    print(elem)
    copyB.remove(elem) # IF CELL ID 
unit_to_keep = copyB

A_upd = A.loc[unit_to_keep,:,:]
C_upd = C.loc[unit_to_keep,:]
S_upd = S.loc[unit_to_keep,:]

TodropFile = folderMouse / f'TodropFileAB.json'

with open(TodropFile, 'w') as f:
    json.dump(unit_to_drop, f, indent=2) 

Extract Ca2+ peaks

In [ ]:
## For one neuron, need to be implemented for the whole set and concatenate the np into one xarray with 

Indiv_trace = C_upd[2,:].to_series()
peaks, properties = find_peaks(Indiv_trace)
# peak boundaries taken at 70% from peak of intensity. This means that the peaks with small amplitude will be longer than the big ones.
results_width = peak_widths(Indiv_trace, peaks, rel_height=0.7)
# Organise results in numpy array
peaks2 = peaks.reshape(len(peaks),1)
npresults_width = np.array(results_width).reshape(4,-1)
peak_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()


In [ ]:
for i in range(len(unit_to_keep)):
    Indiv_trace = C_upd[i,:].to_series()
    peaks, properties = find_peaks(Indiv_trace)
# peak boundaries taken at 70% from peak of intensity. This means that the peaks with small amplitude will be longer than the big ones.
    results_width = peak_widths(Indiv_trace, peaks, rel_height=0.7)
# Organise results in numpy array
    peaks2 = peaks.reshape(len(peaks),1)
    npresults_width = np.array(results_width).reshape(4,-1)
    peak_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()

In [ ]:
for i in range((unit)):
    file_path=folderMouse
    subfolder = file_path.parents[1].stem
    if subfolder == 'continuous':
        recording = file_path.parents[2].stem.replace('recording','')
        print(recording)
        file = file_path.stem
        print(recording, file)
        np_arr = np.load(file_path)
        datalen = len(np_arr)
        print(file, datalen)
        if recording == 1: #not in TTL_stamp2:
            TTL_stamp2.append(recording)
            coords = {
                'channels' : np.array(['synchronized_timestamps', 'timestamps']),
                'duration_rec' : np.arange(datalen)
            }
            globals()[f"StampsCont_{recording}"] = xr.DataArray(coords=coords, dims=['channels', 'duration_rec'])
        globals()[f"StampsCont_{recording}"].loc[file,:] = np_arr   


In [ ]:
"""
MapSessionsPath = folderMouse.parents[2] / f'mappings.pkl'
MapSessions = pd.read_pickle(MapSessionsPath)
MapSessions
"""